# Set file paths

In [1]:
SERGIO_HOME = '/scratch/ch153/packages/SERGIO/PayamDiba/SERGIO'
SERGIO_DATASETS_DIR = '/scratch/ch153/packages/SERGIO/PayamDiba/SERGIO/data_sets'
INPUTS_BEELINE_DIR = '/scratch/ch153/packages/BEELINE/hlab1/Beeline/inputs_beeline3'

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import shutil
sys.path.append(SERGIO_HOME)
from SERGIO.sergio import sergio

# Test dropout calculation for DS6 net0

In [3]:
exprU_clean = pd.read_csv(os.path.join(SERGIO_DATASETS_DIR,'De-noised_100G_6T_300cPerT_dynamics_7_DS6/simulated_noNoise_U_0.csv'), 
                                       sep=',', header=0, index_col=0)
exprS_clean = pd.read_csv(os.path.join(SERGIO_DATASETS_DIR,'De-noised_100G_6T_300cPerT_dynamics_7_DS6/simulated_noNoise_S_0.csv'), 
                                       sep=',', header=0, index_col=0)


In [4]:
exprU = np.stack(np.split(exprU_clean,6,axis=1))
exprS = np.stack(np.split(exprU_clean,6,axis=1))
print(exprU.shape)
print(exprS.shape)

(6, 100, 300)
(6, 100, 300)


In [5]:
df = pd.read_csv(os.path.join(SERGIO_DATASETS_DIR,'De-noised_100G_6T_300cPerT_dynamics_7_DS6/bMat_cID7.tab'), 
                 sep='\t', header=None, index_col=None)
bMat = df.values

sim = sergio(number_genes=100, number_bins = 6, number_sc = 300, noise_params = 0.2, decays=0.8, sampling_state = 1, noise_params_splice = 0.07, noise_type='dpd', dynamics=True, bifurcation_matrix= bMat)
sim.build_graph(input_file_taregts =os.path.join(SERGIO_DATASETS_DIR,'De-noised_100G_6T_300cPerT_dynamics_7_DS6/Interaction_cID_7.txt'), 
                input_file_regs=os.path.join(SERGIO_DATASETS_DIR,'De-noised_100G_6T_300cPerT_dynamics_7_DS6/Regs_cID_7.txt'), 
                shared_coop_state=2)

binary_indU, binary_indS = sim.dropout_indicator_dynamics(exprU, exprS, shape = 6.5, percentile = 82)
exprU_D = np.multiply(binary_indU, exprU)
exprS_D = np.multiply(binary_indS, exprS)
exprT_D = exprU_D + exprS_D

In [6]:
exprU_df = pd.DataFrame(np.concatenate(exprU_D,axis=1))
exprS_df = pd.DataFrame(np.concatenate(exprS_D,axis=1))
exprT_df = pd.DataFrame(np.concatenate(exprT_D,axis=1),columns=np.repeat(range(6),300))
exprU_df.to_csv('exprU_df.csv',sep=',')
exprS_df.to_csv('exprS_df.csv',sep=',')
exprT_df.to_csv('exprT_df.csv',sep=',')

In [7]:
exprU_df.iloc[4,3:6]

3    0.000000
4    6.760398
5    0.000000
Name: 4, dtype: float64

In [8]:
"""
Convert to UMI count
"""
count_matrix_U, count_matrix_S = sim.convert_to_UMIcounts_dynamics(exprU_D, exprS_D)

"""
Make 2d spliced and unspliced expression matrices
"""
count_matrix_U = np.concatenate(count_matrix_U, axis = 1)
count_matrix_S = np.concatenate(count_matrix_S, axis = 1)
count_matrix_T = count_matrix_U + count_matrix_S

# Copy source datasets

In [9]:
SERGIO_DATASET_PARAMS = pd.DataFrame(data={
    'exp_name':['DS1','DS2','DS3','DS8','DS4','DS5','DS6','DS7'],
    'exp_dir':['SERGIO_DS1','SERGIO_DS2','SERGIO_DS3',
               'SERGIO_DS8','SERGIO_DS4','SERGIO_DS5',
               'SERGIO_DS6','SERGIO_DS7'],
    'source_dir':['De-noised_100G_9T_300cPerT_4_DS1',
                  'De-noised_400G_9T_300cPerT_5_DS2',
                  'De-noised_1200G_9T_300cPerT_6_DS3',
                  'De-noised_100G_3T_300cPerT_dynamics_8_DS8',
                  'De-noised_100G_3T_300cPerT_dynamics_9_DS4',
                  'De-noised_100G_4T_300cPerT_dynamics_10_DS5',
                  'De-noised_100G_6T_300cPerT_dynamics_7_DS6',
                  'De-noised_100G_7T_300cPerT_dynamics_11_DS7'],
    'expfile':['simulated_noNoise_{network_i}.csv',
               'simulated_noNoise_{network_i}.csv',
               'simulated_noNoise_{network_i}.csv',
               'simulated_noNoise_{sp}_{network_i}.csv',
               'simulated_noNoise_{sp}_{network_i}.csv',
               'simulated_noNoise_{sp}_{network_i}.csv',
               'simulated_noNoise_{sp}_{network_i}.csv',
               'simulated_noNoise_{sp}_{network_i}.csv'],
    'bmatfile':np.repeat(['bMat_cID{cid}.tab'],8),
    'interactionfile':np.repeat(['Interaction_cID_{cid}.txt'],8),
    'netfile':np.repeat(['gt_GRN.csv'],8),
    'regsfile':np.repeat(['Regs_cID_{cid}.txt'],8),
    'nsim':np.repeat([15],8),
    'cid':['4','5','6','8','9','10','7','11'],
    'ncells_per_cl':[300,300,300,300,300,300,300,300],
    'ngenes':[100,400,1200,100,100,100,100,100],
    'nclusters':[9,9,9,3,3,4,6,7],
    'desc':['100G_9T','400G_9T','1200G_9T','3T_BIF',
            '3T_LIN','4T_BIF','6T_TRIF','7T_TREE']})

In [10]:
for params_i in range(4,SERGIO_DATASET_PARAMS.shape[0]):
    print(params_i)
    params_row = SERGIO_DATASET_PARAMS.iloc[params_i]
    
    os.makedirs(os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir']), exist_ok = True)
    for fname in ['bmatfile','interactionfile','netfile','regsfile']:
        fpath = params_row[fname].format(cid=params_row['cid'])
        print(fpath)
        shutil.copyfile(os.path.join(SERGIO_DATASETS_DIR,params_row['source_dir'],fpath),
                        os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],fpath))
    
    for network_i in range(params_row['nsim']):
        for sp in ['U','S','T']:
            sim_in = params_row['expfile'].format(sp=sp,network_i=network_i)
            sim_out_dir = os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],'net{network_i}'.format(network_i=network_i))
            os.makedirs(sim_out_dir, exist_ok = True)
            shutil.copy(os.path.join(SERGIO_DATASETS_DIR,params_row['source_dir'],sim_in),
                        os.path.join(sim_out_dir,sim_in))

4
bMat_cID9.tab
Interaction_cID_9.txt
gt_GRN.csv
Regs_cID_9.txt
5
bMat_cID10.tab
Interaction_cID_10.txt
gt_GRN.csv
Regs_cID_10.txt
6
bMat_cID7.tab
Interaction_cID_7.txt
gt_GRN.csv
Regs_cID_7.txt
7
bMat_cID11.tab
Interaction_cID_11.txt
gt_GRN.csv
Regs_cID_11.txt


# Generate drop-out data

In [11]:
dropout_shape = [6.5]
dropout_perc = [80]
for params_i in range(4,SERGIO_DATASET_PARAMS.shape[0]):
    params_row = SERGIO_DATASET_PARAMS.iloc[params_i]
    print(params_row)

    bmatfile = pd.read_csv(os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],params_row['bmatfile'].format(cid=params_row['cid'])),
                 sep='\t', header=None, index_col=None)
    bMat = df.values
    sim = sergio(number_genes=params_row['ngenes'], number_bins = params_row['nclusters'], number_sc = params_row['ncells_per_cl'], 
                 noise_params = 0.2, decays=0.8, sampling_state = 1, noise_params_splice = 0.07, noise_type='dpd', 
                 dynamics=True, bifurcation_matrix= bMat)
    sim.build_graph(input_file_taregts =os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],params_row['interactionfile'].format(cid=params_row['cid'])), 
                    input_file_regs=os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],params_row['regsfile'].format(cid=params_row['cid'])), 
                    shared_coop_state=2)

    print('Networks')
    for network_i in range(15):
        print('{network_i}'.format(network_i=network_i),end=" ")
        exprU_clean = pd.read_csv(os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],'net{network_i}'.format(network_i=network_i),
                                               params_row['expfile'].format(sp='U',network_i=network_i)),
                                               sep=',', header=0, index_col=0)
        exprS_clean = pd.read_csv(os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],'net{network_i}'.format(network_i=network_i),
                                               params_row['expfile'].format(sp='S',network_i=network_i)),
                                               sep=',', header=0, index_col=0)
        # reshape (#genes * (#cells_per_type * #cell_types)) to (#cell_types * #genes * #cells_per_type) 
        exprU = np.stack(np.split(exprU_clean,params_row['nclusters'],axis=1))
        exprS = np.stack(np.split(exprS_clean,params_row['nclusters'],axis=1))
        
        # add drop outs
        for dsh in dropout_shape:
            for dp in dropout_perc:
                binary_indU, binary_indS = sim.dropout_indicator_dynamics(exprU, exprS, shape = dsh, percentile = dp)
                exprU_D = np.multiply(binary_indU, exprU)
                exprS_D = np.multiply(binary_indS, exprS)
                exprT_D = exprU_D + exprS_D

                # reshape to (#genes * (#cells_per_type * #cell_types)) and write to csv
                exprU_df = pd.DataFrame(np.concatenate(exprU_D,axis=1))
                exprS_df = pd.DataFrame(np.concatenate(exprS_D,axis=1))
                exprT_df = pd.DataFrame(np.concatenate(exprT_D,axis=1),columns=np.repeat(range(params_row['nclusters']),params_row['ncells_per_cl']))

                out_dir  = os.path.join(INPUTS_BEELINE_DIR,params_row['exp_dir'],'net{network_i}_sh{dsh}_perc{dp}'.format(network_i=network_i,dsh=dsh,dp=dp))
                os.makedirs(out_dir, exist_ok = True)            
                exprU_df.to_csv(os.path.join(out_dir,'simulated_noNoise_U_D.csv'),sep=',')
                exprS_df.to_csv(os.path.join(out_dir,'simulated_noNoise_S_D.csv'),sep=',')
                exprT_df.to_csv(os.path.join(out_dir,'simulated_noNoise_T_D.csv'),sep=',')
    print("")

exp_name                                                 DS4
exp_dir                                           SERGIO_DS4
source_dir         De-noised_100G_3T_300cPerT_dynamics_9_DS4
expfile               simulated_noNoise_{sp}_{network_i}.csv
bmatfile                                   bMat_cID{cid}.tab
interactionfile                    Interaction_cID_{cid}.txt
netfile                                           gt_GRN.csv
regsfile                                  Regs_cID_{cid}.txt
nsim                                                      15
cid                                                        9
ncells_per_cl                                            300
ngenes                                                   100
nclusters                                                  3
desc                                                  3T_LIN
Name: 4, dtype: object
Networks
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
exp_name                                                  DS5
exp_dir         